# Decorators
<!-- > "Using decorator to document sample selection process." -->

- toc: true
- badges: true
- comments: true
- categories: [python]

### Decorators with arguments

I have a simple logger decorator.

In [28]:
def logger(func):
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls
        calls +=1
        print(f'Call #{calls} of {func.__name__}')
        return func(*args, **kwargs)
    return wrapper

@logger
def greeter():
    print('Hello')

@logger
def singer():
    print('lalala')
    
@logger
def congratulator():
    print('Congratulations!')
    
greeter()
greeter()
singer()
congratulator()

Call #1 of greeter
Hello
Call #2 of greeter
Hello
Call #1 of singer
lalala
Call #1 of congratulator
Congratulations!


Now I want the ability to deactivate the logger for certain functions. So I wrap the decorator in a decorator factory, like so:

In [29]:
def param_logger(active=True):
    def decorator(func):
        calls = 0
        def wrapper(*args, **kwargs):
            nonlocal calls
            if active:
                calls +=1
                print(f'Call #{calls} of {func.__name__}')
            return func(*args, **kwargs)
        return wrapper
    return decorator

@param_logger()
def greeter():
    print('Hello')
    
@param_logger(active=True)
def singer():
    print('lalala')    

@param_logger(active=False)
def congratulator():
    print('Congratulations!')
    
greeter()
greeter()
singer()
congratulator()

Call #1 of greeter
Hello
Call #2 of greeter
Hello
Call #1 of singer
lalala
Congratulations!


How does this work? I'm not completely confident, actually, but this is how I explain it to myself.

In our initial logger function above, both the argument to the outer function  (*func*) and the variable defined inside the outer function (*calls*) are free variables of the closure function wrapper, meaning that wrapper has access to them even though they are not bound inside wrapper.

If we remember that 

In [216]:
@logger
def singer():
    print('lalala')

is equivalent to

In [218]:
singer = logger(singer)

then it's clear that we can get a view of the free variables of the decorated greeter variable like so:

In [201]:
logger(greeter).__code__.co_freevars

('calls', 'func')

Now, what are the free variables of param_logger?

In [202]:
param_logger().__code__.co_freevars

('active',)

This makes sense: *active* is the function argument and we do not define any additional variables inside the scope of param_logger, so given our result above, this is what we would expect.

But param_logger is a decorator factory and not a decorator, which means it produces a decorator at the time of decoration. So, what are the free variables of the decorator is produces?

Similar to above, remembering that

In [219]:
@param_logger()
def singer():
    print('lalala')

is equivalent to

In [220]:
singer = param_logger()(singer)

we can inspect the decorated singer function's free variables like so:

In [222]:
param_logger()(singer).__code__.co_freevars

('active', 'calls', 'func')

We can see that active is now an additional free variable that our wrapper function has access to, which provides us with the answer to our question: decorator factories work by producing decorators at decoration time and passing on the specified keyword to the decorated function.

A final point for those into aesthetics or coding consistency: we can tweak our decorator factory so that we can ommit the () if we pass no keyword arguments.

In [245]:
def logger(func=None, active=True):
    def decorator(func):
        calls = 0
        def wrapper(*args, **kwargs):
            nonlocal calls
            if active:
                calls +=1
                print(f'Call #{calls} of {func.__name__}')
            return func(*args, **kwargs)
        return wrapper
    if func:
        return decorator(func)
    else:
        return decorator
    
@logger
def greeter():
    print('Hello')
    
@logger()
def babler():
    print('bablebalbe')   

@logger(active=True)
def singer():
    print('lalala')   

@logger(active=False)
def congratulator():
    print('Congratulations!')
    
greeter()
greeter()
babler()
singer()
congratulator()

Call #1 of greeter
Hello
Call #2 of greeter
Hello
Call #1 of babler
bablebalbe
Call #1 of singer
lalala
Congratulations!


To understand what happens here, remember that decorating *func* with a decorator is equivalent to

In [ ]:
func = decorator(func)

While decorating it with a decorator factory is equivalent to

In [ ]:
func = decorator()(func)

The control flow inside the above decorator factory simply switches between these two cases: if logger gets a function argument, then that's akin to the first scenario, where the func argument is passed into decorator directly, and so the decorator factory returns *decorator(func)* to mimik this behaviour. If *func* is not passed, then we're in the standard decorator factory scenario above, and we simply return the decorator uncalled, just as any plain decorator factory would. 

Recipe 9.6 in the [Python Cookbook](https://www.oreilly.com/library/view/python-cookbook-3rd/9781449357337/) discusses a neat solution to the above for a registration decorator using functools.partial(), which I haven't managed to a scenario with a decorator factory. Might give it another go later.

### Mistakes I often make

I often do the below:

In [8]:
from functools import wraps

def decorator(func):
    @wraps
    def wrapper(*args, **kwargs):
        print('Func is called:', func.__name__)
        return func(*args, **kwargs)
    return wrapper

@decorator
def greeter(name):
    return f'Hello {name}'

greeter('World')

AttributeError: 'str' object has no attribute '__module__'

What's wrong, there? `@wraps` should be `@wraps(func)`.

In [9]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Func is called:', func.__name__)
        return func(*args, **kwargs)
    return wrapper

@decorator
def greeter(name):
    return f'Hello {name}'

greeter('World')

Func is called: greeter


'Hello World'

## Main sources

- [Fluent Python](https://www.oreilly.com/library/view/fluent-python/9781491946237/)
- [Python Cookbook](https://www.oreilly.com/library/view/python-cookbook-3rd/9781449357337/)
- [Learning Python](https://www.oreilly.com/library/view/learning-python-5th/9781449355722/)